***GENERATED CODE FOR t111 PIPELINE***

**CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM**

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(inStages, inStagesData, stageId, spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(inStages, inStagesData, stageId, spark, config):
        return inStagesData.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                                        delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


**TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA**

In [ ]:
import pyspark
import json
from pyspark.sql.functions import round
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationTimeSeriesForecastingMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]['featureList']
        ForecastFE = configObj["FE"]
        featuresSelectedList = [ForecastFE['features']
                                ['timecolumn'], ForecastFE['features']['tocompare']]
        transformedDF = transformationDF.select(
            [c for c in transformationDF.columns if c in featuresSelectedList])
        transformedDF = CleanseData().replaceNullValues(featureData, transformedDF)
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'y', 'threshold': '8', 'transformation_label': 'Binarizer'}], 'feature': 'y', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
                                              'count': '297', 'mean': '8.13', 'stddev': '0.76', 'min': '6.79122146272619', 'max': '11.9767789709185', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}]}, {'feature_label': 'y', 'threshold': '8', 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('y')

        transformedDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
        df = transformedDF.toPandas()
        return df


**AUTOML FUNCTIONS**

In [ ]:
from fbprophet import Prophet
import pandas as pd


def driverProphet(df):
    df.columns = ['ds', 'y']
    df['ds'] = df['ds'].astype(str).str[:-6]
    df['ds'] = pd.to_datetime(df['ds'])
    m = Prophet()
    m.fit(df)
    future = m.make_future_dataframe(periods=365)
    forecast = m.predict(future)
    forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
    m.plot(forecast)
    return m


**READING DATAFRAME**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

try: 
	t111dbfs = DBFSConnector.fetch([], {}, "5ec56b7d7e4107042c48a30e", spark, "{'url': '/Demo/example_wp_log_peyton_manning.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")

except Exception as ex: 
	logging.error(ex)


**TRANSFORMING DATAFRAME**

In [ ]:
try: 
	t111featureforecast = TransformationTimeSeriesForecastingMain.run(t111dbfs, json.dumps( {"FE": {"functionList": [{"function": "log"}], "featureList": [{"transformationsData": [{}], "feature": "ds", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "generated": "True", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "y", "threshold": "8", "transformation_label": "Binarizer"}], "feature": "y", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "297", "mean": "8.13", "stddev": "0.76", "min": "6.79122146272619", "max": "11.9767789709185", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}]}, {"feature": "y_binarizer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "selected": "True", "transformationsData": [{}], "type": "real", "stats": {"count": "297", "mean": "0.48", "stddev": "0.5", "min": "0.0", "max": "1.0", "missing": "0"}}], "originalfile": "/Demo/example_wp_log_peyton_manning.csv", "features": {"timecolumn": "ds", "tocompare": "y"}, "dataPercentage": "30", "statFunction": {"function": "Original", "parameter": ""}}}))

except Exception as ex: 
	logging.error(ex)


**TRAIN MODEL**

In [ ]:
try: 
	model = driverProphet(t111featureforecast)

except Exception as ex: 
	logging.error(ex)


**PREDICT ON TRAINED MODEL**

In [ ]:

try:
        future = model.make_future_dataframe(periods=30,freq='MS',include_history=False)
        predictedDataFrame = model.predict(future)
        display(model.plot(predictedDataFrame))
        display(model.plot_components(predictedDataFrame))
        display(predictedDataFrame)
except Exception as ex:
    logging.error(ex)

